<h3>Analysis of Yearly NYC Subway Station Usage for 2013 </h3>

<h6>Edwin Heredia, edw.delta@mail.com, July 2016</h6>


The data set consits of entry and exit counters measured every four hours at every turnstile available in stations in the NYC Subway System. <p>

This document computes the station usage per month. For each month, it aggregates the turnstile busy scores. 

<b>Busy Score</b>
Let $ent_i(n)$ and $ext_i(n)$ represent the $i$th turnstile entry and exit counters at time $n$. Let $I = [n,m]$ represent a time interval between a time value $n$ and a time value $m$, with $m > n$.  Then the $i$th turnstile busy score for an interval $I$ is given by:<p>
<center>
$b_i(n,m) = [ent_i(m) - ent_i(n)] + [ext_i(m) - ext_i(n)]$
</center>

<b>Reset condition</b> The data shows that sometimes the entry or exit counters at time $m$ are smaller than those at time $n$. This issue implies that for some reason the counters were reset. The data analysis program excludes any records that show indication of a reset condition. 

<b>Station Usage</b> Consider the set of turnstile machines that belong to some station $sta_j$. The station usage $u_j(n,m)$ for a time interval $I = [n,m]$ is defined as the aggregation of busy scores during $I$ for all turnstiles that belong to the station. That is,<p>
<center>
$u_j(n,m) = \sum_{all \;i \;in \;sta_j} b_i(n,m)$
</center>

<b>Yearly usage growth rate</b>
This document computes station usage per month during 2013. For each station, the resulting time series can be modeled using linear regression as $u = a k + b$ where $u$ is the station montly usage and $k$ is the month (a value between 1 and 12). The term $a$ is the slope and $b$ is the intercept. We use $a$ as a mesure of yearly usage growth. This document computes the station with the largest usage growth and the station with the largest decline.<p>

<b>Data sources</b>
The format of the data for the 2013 measurements is described here: http://web.mta.info/developers/resources/nyct/turnstile/ts_Field_Description_pre-10-18-2014.txt <p>

The following link provides a list of the web files containing the data: http://web.mta.info/developers/turnstile.html 


<h4>Process and Software Description</h4>

The <b>get_url()</b> and <b>get_month()</b> utility function provide respectively: A URL for the raw-data file containing the starting date/time for a given month, and the index of the month when representing months as integers between 1 and 12. 

In [2]:
import urllib2
import pandas as pd
import numpy as np
import statsmodels.api as smapi

def get_url(month):
    """Returns the URL for the starting date/time of a particular month. A month is defined by a 3-letter code."""
    monthdata = {"jan": "130105.txt", "feb": "130202.txt", "mar": "130302.txt", "apr": "130406.txt",
                 "may": "130504.txt", "jun": "130608.txt", "jul": "130706.txt", "aug": "130803.txt",
                 "sep": "130907.txt", "oct": "131005.txt", "nov": "131102.txt", "dec": "131207.txt",
                 "jan_next": "140104.txt"}

    prefix = "http://web.mta.info/developers/data/nyct/turnstile/turnstile_"

    file = monthdata.get(month, None)

    if file is not None:
        return prefix + file
    else:
        return None


def get_month(index):
    """Returns a 3-letter string for a month identified by an index whose value is between 1 and 12"""
    months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug",
              "Sep", "Oct", "Nov", "Dec"]

    if index < 1 or index > 12:
        return None
    else:
        return months[index - 1]


The <b>filtered_dataframe()</b> function parses a raw-data file defined by a URL and returns a pandas dataframe. The function finds turnstile records for a particular day and time. The day and time input arguments use the same format as the one used by the raw-data file.<p> 
The function returns a collection of records (rows) where each record describes information for a single turnstile. A turnstile is defined by the station's identifier "ca", the unit identifier "unit", and the machine identifier "scp". The record includes the target month, day, and year as columns using integer values. The record includes the entry and exit counter values at the target date. 

In [3]:
def filtered_dataframe(target_url, day, time, num_lines):
    """Parses a raw data file defined by a URL. It returns a dataframe with all the turnstile records
    that match a certain day  and time. The day and time arguments use the format defined in the raw-data 
    files. The argument num_lines is used to control how many lines in the raw-data file should be parsed. 
    If num_lines is set to -1 it indicates all lines. 
    
    The program rejects turnstile records whose entry or exit counters show negative numbers 
    """
    webdata = urllib2.urlopen(target_url)

    filtered_data = []
    line_counter = 0
    for line in webdata:
        values = line.strip().split(",")

        temp_array = convert_to_array2(values, day, time)


        filtered_data.extend(temp_array)

        if num_lines != -1 and line_counter >= num_lines:
            break
        else:
            line_counter += 1

    print "[Info] Total number of scanned lines: ", line_counter

    tempdf1 =  pd.DataFrame(filtered_data,
                            columns=["ca", "unit", "scp", "month", "day", "year", "hour", "ent", "exit" ])

    # Remove records that for some unknown reason have a negative counter
    tempdf2 = tempdf1[tempdf1["ent"] > 0]
    tempdf3 = tempdf2[tempdf2["exit"] > 0]

    return tempdf3


def convert_to_array2(values, day, time):
    """Utility function that works together with the filtered_dataframe() function"""
    reduced = []
    sta = values[0]
    unit = values[1]
    scp = values[2]

    for i in range(3, len(values), 5):
        vec = values[i:i+5]

        if vec[0] == day and vec[1] == time and vec[2] == 'REGULAR':
            monval, dayval, yearval = [int(val) for val in day.split("-")]
            timeval = int(time.split(":")[0])
            filtered_data = [sta, unit, scp, monval, dayval, yearval, timeval, int(vec[3]), int(vec[4])]

            reduced.append(filtered_data)

    return reduced


The <b>merge_init_final()</b> function merges turnstile data for an initial day/time with turnstile data for a final day/time. This function then computes the busy score for every turnstile.<p> 

The merge procedure uses the turnstile identifiers "ca", "unit", and "scp" as the key. In other words, the process identifies a turnstile and merges the columns extracted from the input data: initial and final dataframes. In the process, any turnstile that does not appear in either the initial or final dataframes is removed from the merged table. <p>

This function computes the busy score according to the equation defined in the introductory section of this document. During the calculation, the function checks to see if there is a reset condition. If any of the counters (entry or exit) for a turnstile have been reset, then the function removes the turnstile record from the results.<p>

This function checks to see if entry or exit counters are negative. In this case, the corresponding turnstile is also removed from the results. 

In [4]:
def merge_init_final(init_df, final_df, show=False):
    """
    Merges two data frames using 'ca', 'unit' and 'scp' as the key. Each data frame represents the
    entry and exit measurements at a specific day and time for all turnstile machines and all stations.
    The 'show' argument controls a verbose output
    """

    # Merging the initial and final data sources using 'ca', 'unit', and 'scp' as the key.

    if show:
        print "** Merging data frames that have the following sizes: ", init_df.shape, final_df.shape
        print "** INITIAL FRAME: ............."
        print init_df
        print "** ENDING FRAME: .............."
        print final_df

    mdf = pd.merge(init_df, final_df, on=["ca", "unit", "scp"])

    # Compute the difference in entry counters and exit counters. Also compute busy score
    mdf["ent_diff"] = mdf["ent_y"] - mdf["ent_x"]
    mdf["exit_diff"] = mdf["exit_y"] - mdf["exit_x"]
    mdf["busy"] = mdf["ent_diff"] + mdf["exit_diff"]

    # Process machines whose entry counts during the interval show negative values
    reset1 = mdf[mdf["ent_diff"] < 0]
    if len(reset1) > 0:
        print "[Note] Machines that had a reset of the entry counter. Data for these machines will be removed:"
        print reset1[["ca", "unit", "scp", "ent_x", "ent_y"]]
        mdf2 = mdf[mdf["ent_diff"] > 0]
    else:
        mdf2 = mdf

    # Process machines whose exit counts during the interval show negative values
    reset2 = mdf2[mdf2["exit_diff"] < 0]
    if len(reset2) > 0:
        print "[Note] Remaining machines showing a reset of the exit counter. Data for these machines will be removed:"
        print reset2[["ca", "unit", "scp", "exit_x", "exit_y"]]
        mdf3 = mdf2[mdf2["exit_diff"] > 0]
    else:
        mdf3 = mdf2

    return mdf3


The <b>retrieve_stations()</b> function is used to generate a list of stations extracted from a raw-data file. The raw-data file is defined by a URL. The 'stations' argument is a dictionary that may be empty or already include some stations. If the latter is true, the function adds more data to the dictionary. <p> 

The stations dictionary uses the station identifier as key (the station identifier is known as 'ca' in the raw-data file). The value for each key is a counter of records. In other words, the value counts how many times a station has been found as a record in a raw-data file. <p>

The num_lines argument indicates how many lines should be parsed in the target raw-data file. A value of -1 indicates all lines. 

In [5]:
def retrieve_stations(target_url, stations, num_lines):
    """Retrieves a list of unique stations from a raw data file defined by its URL. Stores the
    results in the 'stations' dictionary. If the dictionary is not empty it will add information
    to the dictionary. The num_lines argument controls the number of parsed lines in the raw
    data files. A value of num_lines=-1 indicates all lines."""
    webdata = urllib2.urlopen(target_url)

    if num_lines < -1:
        num_lines == -1

    line_count = 0
    for line in webdata:
        values = line.strip().split(",")
        stations[values[0]] = stations.get(values[0], 0) + 1
        line_count += 1
        if line_count != -1 and line_count == num_lines:
            break

The <b>station_busy_score_per_month()</b> function computes the busy score per station and per month as defined in the introductory section of the document. The function computes the score for all months in 2013. <p>

Therefore the time interval in this case is one month defined by two dates: the first day of the month at 00:00:00 and the first day of the next month at 00:00:00. There is only one month that breaks this rule. For June 2013 the function uses the second day instead of the first. This is due to raw-files that split the records for one day in two different files. We avoid this problem by simply using the second day instead of trying to merge data from two files. <p>

The function returns a 12-element array of dataframes (one dataframe per month). Each dataframe includes the list of stations and their usage scores. 

<b>The Daylight Savings Time (DST) problem</b> Running this function reveals that March and November have a very low number of records for participating stations. Upon further examination, we discovered that this is caused by the time shift due to DST. While every day turnstile records are collected at 00:00:00, during these months many records show the collection time at 01:00:00. Currently the <b>filtered_dataframe()</b> parsing function does not compensate for this issue. In the future, a more complex implementation of the parsing function could be designed to extract records taking into account the exact day when DST starts and ends. 

In [6]:
def station_busy_score_per_month(lines):
    """Computes how busy a station is on a per-month basis. The argument 'lines' indicates the
    number of lines of the raw data files that should be examined. A value of lines=-1 indicates
    all lines.
    """
    months = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug',
              'sep', 'oct', 'nov', 'dec', 'jan_next']

    # Days of the month for data sampling. Notice that June is sampled on the 2nd to avoid
    # boundary problems in the files.
    indays = ['01', '01', '01', '01', '01', '02', '01', '01', '01', '01', '01', '01', '01']

    dframes = []
    monthframes = []

    for ind, month in enumerate(months):
        # convert months into string representation used in NYC data
        if ind == 12:    # next year
            monthval = '01'
            yearval = str(14)  # year 2014
        else:
            monthval = str(ind + 1) if (ind + 1) >= 10 else '0' + str(ind + 1)
            yearval = str(13)  # year 2013

        # Assemble a date value with format comparable to that used in the data set
        dateval = monthval + "-" + indays[ind] + "-" + yearval

        print "\n---- Requesting subway station data for: ", dateval
        # print "   url: ", get_url(month)

        df = filtered_dataframe(get_url(month), dateval, '00:00:00', lines)
        dframes.append(df)

        print "[Info] For ", dateval, "the number of records in the raw-data file is ", df.shape[0]

    # merge the data from the beginning and the end of the month to compute montly usage
    for ind in range(len(dframes) - 1):

        print "\n---- Merging the starting and ending data for ", get_month(ind)

        merged = merge_init_final(dframes[ind], dframes[ind + 1])

        sta_df = merged[["ca", "busy"]].groupby("ca")["busy"].sum()

        monthframes.append(sta_df)

    return monthframes


The <b>fit_regression_line()</b> function takes a pandas Series representing station usage per month and fits a regression line using a least-squares procedure. The function returns the slope of the fitted line as a measure of a station's yearly usage growth. <p>

Before actually computing a regression line, the function has to deal with three problems:<p>

<ul>
<li> <b>Null usage values</b> When computing the station usage value per month many stations end up with a null usage value. The regression function skips any null values. This happens mainly if the station has data only for the initial time (beginning of the month), only for the final time (beginning of next month), or both. This problem happens often throughout the year, and additionally, it happens specially for data derived from the months of March and November. The months of March and November have non-null records for only a few stations due to the DST problem mentioned earlier. </li>
<li> <b>Stations with data for only a few months</b> Due to null usage values some stations end up having data only for a few months in the year, which could result in poor estimates of usage growth. For this reason, the regression function only computes a slope (usage growth) for a station if the station has data from at least 4 months. </li>
<li> <b>Outliers</b> Some stations like N049, N046, and H19 end up with one month whose usage value is extremely high compared to the other values. For example, if the usage values are in the hundred thousands, the outlier has a value in the order of ten million. The function uses an outlier removal procedure to deal with this problem. For each series representing a station usage in a month, the function computes the dynamic range as a percentage of the minimum value. In other words, it computes the ratio:<p>
<center>
$r = \frac{100*(max - min)}{min}$
</center>
If the ratio $r$ is larger than 500% the algorithm removes the sample with the maximum value because it considers it to be an outlier. </li>
</ul>

In [7]:
def fit_regression_line(ser):
    """
    Fits a regression line using least squares with 'y' as the monthly usage and 'x' as the
    month index (1, 2, ...). The 'ser' parameter is a pandas Series whose index is the month
    index and the value is the montly usage. The 'ser' parameter represents usage data for a
    station over one year

    Returns the slope of the fitted line as a measure of yearly growth
    """
    # Remove months with null usage values (e.g. due to reset conditions, misaligned sampling times, or no data)
    newser = ser[ser.notnull()]

    # Remove outliers - Strategy: remove the max value if r = 100*(max - min)/min  is larger than 500
    # This means that we allow the max value to be of up to 500% of the minimum

    if len(newser) > 3:
        ratio = (newser.max() - newser.min())*100 / (newser.min() + 1)

        if ratio > 500:
            print "Removing outlier: ", newser[newser == newser.max()]
            # print "index of max: ", newser.idxmax()
            newser.drop([newser.idxmax()], inplace=True)

    # Perform a least-squares fit to model the relation between station usage and month
    if len(newser) > 3:   # require at least 4 sample to fit a line
        # Perform a least-squares line fitting using the model y = w x + b
        xdat = np.array(newser.index)
        xdat2 = [[val] for val in xdat]
        xdat2 = smapi.add_constant(xdat2)

        ydat = np.array(newser.values)

        res = smapi.OLS(ydat, xdat2).fit()

        # print "data size for reg: ", len(newser), " reg parameters:", res.params

        # Return the slope of the line as a measure of yearly growth
        return res.params[1]
    else:
        return 0    # Return 0 if there are not enough samples

The following script retrieves data from the original raw-data files and computes the station usage score per month:

In [8]:
series_vec = station_busy_score_per_month(lines=-1)

frame_vec = []
print "\n[Note] Size of the extracted stations per month: "
for ind, mser in enumerate(series_vec):
    print "    Number of stations for ", get_month(ind+1), ": ", mser.shape[0]
    # print "       ", mser.head()

    mf = pd.DataFrame({'station': mser.index, ind+1: mser.values})
    frame_vec.append(mf)

# Create the table with stations busy scores per month
allfr = reduce(lambda left, right: pd.merge(left, right, on="station", how="outer"), frame_vec)

allfr.set_index("station", inplace=True)


---- Requesting subway station data for:  01-01-13
[Info] Total number of scanned lines:  27667
[Info] For  01-01-13 the number of records in the raw-data file is  1479

---- Requesting subway station data for:  02-01-13
[Info] Total number of scanned lines:  30862
[Info] For  02-01-13 the number of records in the raw-data file is  1510

---- Requesting subway station data for:  03-01-13
[Info] Total number of scanned lines:  31206
[Info] For  03-01-13 the number of records in the raw-data file is  1496

---- Requesting subway station data for:  04-01-13
[Info] Total number of scanned lines:  31266
[Info] For  04-01-13 the number of records in the raw-data file is  2210

---- Requesting subway station data for:  05-01-13
[Info] Total number of scanned lines:  29429
[Info] For  05-01-13 the number of records in the raw-data file is  2210

---- Requesting subway station data for:  06-02-13
[Info] Total number of scanned lines:  29640
[Info] For  06-02-13 the number of records in the raw

The following script fits a regression line for the usage data per month. This procedure is performed for every station in the records. Remember that the procedure has been designed to remove three sources of problems: (1) stations with null usage data in a given month, (2) stations that have usage data for only a few months, and (3) stations whose usage data exhibit a large outlier. <p>

The script reports the list of stations with the highest yearly growth rate. For the top-rated station, the script displays the usage data per month. <p>

The script also reports the list of stations with the lowest yearly growth rates. For the stations with the least growth rate, the script displays the usage data per month<p>

In [9]:
print "\n[Note] Starting the process of fitting a regression line for usage (busy score) as a "
print "function of month. This process computes a regression line for every station with more "
print "than 3 valid monthly measurements. "
print "\n[Note] The slope of this regression line defines the station's growth rate."
slopes = allfr.apply(fit_regression_line, axis=1)


print "\n-----------------------------------------------------"
print "  Subway stations with highest growth rates for 2013 "
print "-----------------------------------------------------"
results = slopes.sort_values(ascending=False)
print results.head()

# print results.head()[0]
top_sta = results.head().index[0]

print "\nUsage data for the station with the highest growth rate: "
for ind, mser in enumerate(series_vec):
    available = True
    try:
        usageval = mser[top_sta]
    except KeyError:
        available = False

    if available:
        print "month: " + get_month(ind+1), "  usage: ", mser[top_sta]

print "\n-----------------------------------------------------"
print "  Subway stations with lowest growth rates for 2013 "
print "-----------------------------------------------------"

results2 = slopes.sort_values(ascending=True)
print results2.head()

lwst_sta = results2.head().index[0]

print "\nUsage data for the station with the lowest growth rate: "
for ind, mser in enumerate(series_vec):
    available = True
    try:
        usageval = mser[lwst_sta]
    except KeyError:
        available = False

    if available:
        print "month: " + get_month(ind + 1), "  usage: ", mser[lwst_sta]




[Note] Starting the process of fitting a regression line for usage (busy score) as a 
function of month. This process computes a regression line for every station with more 
than 3 valid monthly measurements. 

[Note] The slope of this regression line defines the station's growth rate.
Removing outlier:  6    6248444.0
Name: H019, dtype: float64
Removing outlier:  9    44745766.0
Name: N046, dtype: float64
Removing outlier:  9    953879990.0
Name: N049, dtype: float64
Removing outlier:  10    47387.0
Name: N181A, dtype: float64
Removing outlier:  7    125042.0
Name: N191, dtype: float64
Removing outlier:  10    130469.0
Name: N192, dtype: float64
Removing outlier:  9    31740.0
Name: N193, dtype: float64
Removing outlier:  10    46461.0
Name: N194, dtype: float64
Removing outlier:  10    92542.0
Name: N195, dtype: float64
Removing outlier:  5    62217558.0
Name: N325A, dtype: float64
Removing outlier:  8    67129037.0
Name: N330C, dtype: float64
Removing outlier:  10    201101590.0
Na